In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/mathco-thon/train.csv')
test = pd.read_csv('/kaggle/input/mathco-thon/test.csv')
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
data = pd.concat([train,test], axis=0).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data['Engine volume'].value_counts()

In [ ]:
data['Levy'] = data['Levy'].str.replace('-','Missing')

In [ ]:
data['Prod. year'] = data['Prod. year'].astype(object)
data['Engine volume'] = data['Engine volume'].str.replace(' Turbo','').astype(float)
data['Mileage'] = data['Mileage'].str.replace(' km','').astype(int)

In [ ]:
data['Doors'].value_counts()

In [ ]:
data['Doors'] = data['Doors'].str.replace('04-May','4-5').str.replace('02-Mar','2-3')

In [ ]:
data.head()

In [ ]:
train = data.iloc[:19237]
test = data.iloc[-8245:]

In [ ]:
y = np.log1p(train.Price)
features = ['Levy', 'Manufacturer', 'Model', 'Prod. year',
       'Category', 'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags']
X = train[features]


In [ ]:
# import seaborn as sns
# sns.displot(y)

In [ ]:
X['Drive wheels'].value_counts()

In [ ]:
X['Levy'].value_counts()

In [ ]:
X.isnull().sum()

In [ ]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
categorical_cols = X.select_dtypes(include=['category', object]).columns
X = MultiColumnLabelEncoder(columns = categorical_cols).fit_transform(X)
test_data = MultiColumnLabelEncoder(columns = categorical_cols).transform(test[features])

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

car_price_model = RandomForestRegressor(random_state = 1)
car_price_model.fit(train_X, train_y)
car_price_pred = car_price_model.predict(val_X)
from sklearn.metrics import mean_squared_error
mae = mean_squared_error(car_price_pred, val_y)
print(mae)

In [ ]:
test_data

In [ ]:
car_price_model = RandomForestRegressor(random_state = 1)
car_price_model.fit(X, y)

predictions = car_price_model.predict(test_data)

In [ ]:
sample = pd.read_csv('../input/mathco-thon/submission.csv')
sample.head()

In [ ]:
sample['Price'] = np.expm1(predictions)

In [ ]:
sample.describe()

In [ ]:
sample.to_csv('submission_five.csv', index=False)